In [3]:
import pandas as pd
import sys
import os
import re
import nltk
import contractions
import torch
import tensorflow as tf
from datasets import Dataset
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
import numpy as np
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords.add('u')
stopwords.add('th')
sys.path.append("/home/bowenyi/.local/lib/python3.11/site-packages")
!pip install prettytable

2024-01-19 23:35:35.110718: I tensorflow/core/platform/cpu_feature_guard.cc:181] Beginning TensorFlow 2.15, this package will be updated to install stock TensorFlow 2.15 alongside Intel's TensorFlow CPU extension plugin, which provides all the optimizations available in the package and more. If a compatible version of stock TensorFlow is present, only the extension will get installed. No changes to code or installation setup is needed as a result of this change.
More information on Intel's optimizations for TensorFlow, delivered as TensorFlow extension plugin can be viewed at https://github.com/intel/intel-extension-for-tensorflow.
[nltk_data] Downloading package punkt to /home/bowenyi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bowenyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bowenyi/nltk_data...
[nltk_data]   Package stopword

Defaulting to user installation because normal site-packages is not writeable


## 1. Preprocess dataframes

In [4]:
df_before = pd.read_csv("/shared/3/projects/benlitterer/podcastData/processed/beforeFloydMonth/beforeFMonth.tsv", lineterminator = '\n', low_memory=False)
df_in = pd.read_csv("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthEnSHORT.csv", lineterminator = '\n', low_memory=False)

In [5]:
df_before = df_before.dropna(subset=['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10'], how='all')
df_before = df_before.drop_duplicates()
df_before = df_before.drop_duplicates(subset=['potentialOutPath'])

df_in = df_in.dropna(subset=['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10'], how='all')
df_in = df_in.drop_duplicates()
df_in = df_in.drop_duplicates(subset=['potentialOutPath'])

In [5]:
df_before['potentialOutPath'] = df_before['potentialOutPath'].apply(lambda x: "/shared/3/projects/benlitterer/podcastData/prosodyMerged/beforeFMonth" + x)
df_in['potentialOutPath'] = df_in['potentialOutPath'].apply(lambda x: "/shared/3/projects/benlitterer/podcastData/prosodyMerged/floydMonth" + x)

### *1.3 Introduce two columns that represent data labels: is_news and is_politics:* 

In Ashwin's 2021 paper Political Discussion is Abundant..., a podcast is political if it is about political institution (definition modified). Thus, if a podcast is categorized as "government", it is political.

In [6]:
pd.options.mode.chained_assignment = None
df_before['is_news'] = df_before[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']].apply(lambda x: 1 if 'news' in x.values else 0, axis=1)
df_before['is_politics'] = df_before[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']].apply(lambda x: 1 if ('politics' in x.values or 'government' in x.values) else 0, axis=1)

df_in['is_news'] = df_in[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']].apply(lambda x: 1 if 'news' in x.values else 0, axis=1)
df_in['is_politics'] = df_in[['category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10']].apply(lambda x: 1 if ('politics' in x.values or 'government' in x.values) else 0, axis=1)

### *1.4 Introduce three new columns:*
- "transcribed": if a transcript has been transcribed
- "ground_truth_pol": ground truth data on is_political: 0 or 1. Default 2. 
- "grount_truth_news": ground truth data on is_news: 0 or 1. Default 2. 

In [7]:
df_before['transcribed'] = 0
df_before['ground_truth_pol'] = 2
df_before['ground_truth_news'] = 2

df_in['transcribed'] = 0
df_in['ground_truth_pol'] = 2
df_in['ground_truth_news'] = 2

### *1.5 Downsample non-news/non-political podcasts to 4:1*

In [1]:
# Before Floyd Month:
# is_politics:
# df_before_pol = df_before[df_before['is_politics'] == 1].copy()
# n_before_pol = df_before_pol.shape[0]
# df_before_no_pol = df_before[df_before['is_politics'] == 0].sample(n=n_before_pol * 4, replace=False, random_state=387).copy()
# df_before_pol = pd.concat([df_before_pol, df_before_no_pol], ignore_index=True)
# df_before_pol = df_before_pol.sample(frac=1, random_state=387).reset_index(drop=True)

# is_news:
df_before_news = df_before[df_before['is_news'] == 1].copy()
n_before_news = df_before_news.shape[0]
df_before_no_news = df_before[df_before['is_news'] == 0].sample(n=n_before_news * 4, replace=False, random_state=387).copy()
df_before_news = pd.concat([df_before_news, df_before_no_news], ignore_index=True)
df_before_news = df_before_news.sample(frac=1, random_state=387).reset_index(drop=True)

# In Floyd Month:
# is_politics:
# df_in_pol = df_in[df_in['is_politics'] == 1].copy()
# n_in_pol = df_in_pol.shape[0]
# df_in_no_pol = df_in[df_in['is_politics'] == 0].sample(n=n_in_pol * 4, replace=False, random_state=387).copy()
# df_in_pol = pd.concat([df_in_pol, df_in_no_pol], ignore_index=True).sample(frac=1, random_state=387).reset_index(drop=True)

# is_news:
df_in_news = df_in[df_in['is_news'] == 1].copy()
n_in_news = df_in_news.shape[0]
df_in_no_news = df_in[df_in['is_news'] == 0].sample(n=n_in_news * 4, replace=False, random_state=387).copy()
df_in_news = pd.concat([df_in_news, df_in_no_news], ignore_index=True).sample(frac=1, random_state=387).reset_index(drop=True)

NameError: name 'df_before' is not defined

##### Sidenote: 
Pipeline for cross-validation (if used)
1. Split the dataset (non-inFMonth) into 85% for cross-validation (cv) and 15% for test. 
2. Intiate three models (Logistic Regression, Multinomial Naive Bayes, SVM) and a range parameters (RandomSearch)
3. Pick the model and their parameters with the best performance on cv  
3. Train the model on the entire 85% dataset that we use for cv
4. Evaluate the model performance on two test sets: test_cv (15% of the dataset) and test_inFMonth (inFMonth data)

### *1.5 Functions that chunkify transcripts and preprocess the chunks:*  

In [80]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text):
    text = re.sub(r"\n", "", text)  # remove line breaks
    text = re.sub(r'\[.*?\]', '', text)  # remove [Music], (Audio), etc.
    text = re.sub(r'\(.*?\)', '', text)
    text = text.lower()    # convert to lowercase
    text = re.sub(r'\b\w+\.com\b', '', text)  # remove something.com
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)   # remove hyperlinks
    text = re.sub(r"\d+", "", text)   # remove digits and currencies 
    text = re.sub(r'[\$\d+\d+\$]', "", text)
    text = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', text)   # remove dates
    text = re.sub(r'[^\x00-\x7f]', r' ', text)  # remove non-ascii
    text = contractions.fix(text)
    text = re.sub(r'[^\w\s]', '', text)   # remove punctuation

    filtered_tokens = [word for word in word_tokenize(text) if not word in stopwords]
    pos_tags = pos_tag(filtered_tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags if lemmatizer.lemmatize(word, get_wordnet_pos(tag)) not in stopwords]
    lemmatized_text = ' '.join(lemmatized_words)

    return lemmatized_text

# Split transcripts for non-bert models
def split_transcript(transcript):
    transcript['content'] = transcript['content'].fillna('').astype(str)
    start_time = 0
    chunks = []  # List of lists. Each sub-list is a string of chunked text
    chunk = ""
    end_of_sentence = ['.', '!', '?', ']', ')']

    for index, row in transcript.iterrows():
        content = str(row['content'])
        if content.strip() != '':
            if row['end'] - start_time < 60:
                chunk += content
            else:
                chunk += content
                if any(ele in content for ele in end_of_sentence):  # if we reach the end of a sentence
                    chunk = preprocess_text(chunk)
                    if chunk.strip() != '':  # ignore chunks that are only white spaces or empty
                        list = []
                        list.append(chunk)
                        chunks.append(list)
                        start_time = row['end']
                    chunk = ""

    if len(chunk) != 0:
        chunk = preprocess_text(chunk)
        if chunk.strip() != '':
            list = []
            list.append(chunk)
            chunks.append(list)

    return chunks


### *1.6 Obtain beforeFMonth Data and split it into train-dev-test sets*

#### *1.6.1 Non-Bert models: List of features and labels*

In [ ]:
x_before_pol = []
y_before_pol = []
x_before_news = []
y_before_news = []

# is_pol:
for index, row in df_before_pol.iterrows():
    transcript_path = row["potentialOutPath"]
    if os.path.isfile(transcript_path):
        transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
        chunks = split_transcript(transcript)
        if chunks:
            x_before_pol.extend(chunks)
            y_before_pol.extend([row["is_politics"]] * len(chunks))
            row['transcribed'] = 1

# is_news:
for index, row in df_before_news.iterrows():
    transcript_path = row["potentialOutPath"]
    if os.path.isfile(transcript_path):
        transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
        chunks = split_transcript(transcript)
        if chunks:
            x_before_news.extend(chunks)
            y_before_news.extend([row["is_news"]] * len(chunks))
            row['transcribed'] = 1

# Flatten the lists:
x_before_pol = [item for chunk in x_before_pol for item in chunk]
x_before_news = [item for chunk in x_before_news for item in chunk]

In [40]:
'''
Unnecessary when using Keras
'''
# x_train_pol, x_other_pol, y_train_pol, y_other_pol = train_test_split(
#     x_before_pol, y_before_pol, test_size=0.3, random_state=387)

# x_dev_pol, x_test_pol, y_dev_pol, y_test_pol = train_test_split(
#     x_other_pol, y_other_pol, test_size=(2/3), random_state=387)

# x_train_news, x_other_news, y_train_news, y_other_news = train_test_split(
#     x_before_news, y_before_news, test_size=0.3, random_state=387)

# x_dev_news, x_test_news, y_dev_news, y_test_news = train_test_split(
#     x_other_news, y_other_news, test_size=(2/3), random_state=387)

# x_train_pol = [item for sublist in x_train_pol for item in sublist]
# x_dev_pol = [item for sublist in x_dev_pol for item in sublist]
# x_test_pol = [item for sublist in x_test_pol for item in sublist]

# x_train_news = [item for sublist in x_train_news for item in sublist]
# x_dev_news = [item for sublist in x_dev_news for item in sublist]
# x_test_news = [item for sublist in x_test_news for item in sublist]

'\nUnnecessary when using Keras\n'

In [50]:
min_df = 1000 or so (1 or 5% of documents should have that document) 
We can make it more imbalanced: 4 : 1. 4 is non-news, 

count_vect_12 = CountVectorizer(ngram_range=(1, 2))
count_vect_23 = CountVectorizer(ngram_range=(2, 3))

x_12_pol = count_vect_12.fit_transform(x_before_pol)
x_23_pol = count_vect_23.fit_transform(x_before_pol)
x_12_news = count_vect_12.fit_transform(x_before_news)
x_23_news = count_vect_23.fit_transform(x_before_news)

# is_pol:
# ngrams=(1,2):

# x_dev_12_pol = count_vect_12.transform(x_dev_pol)
# x_test_12_pol = count_vect_12.transform(x_test_pol)

# ngrams=(2,3):
# x_dev_23_pol = count_vect_23.transform(x_dev_pol)
# x_test_23_pol = count_vect_23.transform(x_test_pol)

# is_news:
# ngrams=(1,2):
# x_dev_12_news = count_vect_12.transform(x_dev_news)
# x_test_12_news = count_vect_12.transform(x_test_news)

# ngrams=(2,3):
# x_dev_23_news = count_vect_23.transform(x_dev_news)
# x_test_23_news = count_vect_23.transform(x_test_news)

#### *1.6.2 Bert models: Dataframes and datasets/dataDicts*

In [89]:
# Split function for Bert-alike models
def split_transcript_bert(text):
    transcript['content'] = transcript['content'].fillna('').astype(str)
    text = ''
    for ind, row in transcript.iterrows():
        text += str(row['content'])
    return text


df_before_pol_bert = pd.DataFrame(columns=['label', 'text'])
df_before_news_bert = pd.DataFrame(columns=['label', 'text'])

for index, row in df_before_pol.iterrows():
    transcript_path = row["potentialOutPath"]
    if os.path.isfile(transcript_path):
        transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
        label = row['is_politics']
        text = split_transcript_bert(transcript)
        if text:
            df_before_pol_bert.loc[index] = [label, text]
            row['transcribed'] = 1

for index, row in df_before_news.iterrows():
    transcript_path = row["potentialOutPath"]
    if os.path.isfile(transcript_path):
        transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
        label = row['is_news']
        text = split_transcript_bert(transcript)
        if text:
            df_before_news_bert.loc[index] = [label, text]
            row['transcribed'] = 1

ds_before_pol_bert = Dataset.from_pandas(df_before_pol_bert)
ds_before_news_bert = Dataset.from_pandas(df_before_news_bert)

ds_before_pol_bert = ds_before_pol_bert.train_test_split(test_size=0.3, seed=387)
ds_before_news_bert = ds_before_news_bert.train_test_split(test_size=0.3, seed=387)

### *1.7 Obtain inFMonth data (Not now b/c takes too long and not sure of its purpose)*

In [32]:
# x_in_pol = []
# y_in_pol = []
# x_in_news = []
# y_in_news = []

# for index, row in df_in_pol.iterrows():
#     transcript_path = row["potentialOutPath"]
#     if os.path.isfile(transcript_path):
#         transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
#         chunks = split_transcript(transcript)
#         x_in_pol.extend(chunks)
#         y_in_pol.extend([row["is_politics"]] * len(chunks))
#         row['transcribed'] = 1

# for index, row in df_in_news.iterrows():
#     transcript_path = row["potentialOutPath"]
#     if os.path.isfile(transcript_path):
#         transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
#         chunks = split_transcript(transcript)
#         x_in_news.extend(chunks)
#         y_in_news.extend([row["is_news"]] * len(chunks))
#         row['transcribed'] = 1

# x_in_pol = [item for sublist in x_in_pol for item in sublist]
# x_in_news = [item for sublist in x_in_news for item in sublist]

In [ ]:
# df_in_pol_bert = pd.DataFrame(columns=['label', 'text'])
# df_in_news_bert = pd.DataFrame(columns=['label', 'text'])

# for index, row in df_in_pol.iterrows():
#     transcript_path = row["potentialOutPath"]
#     if os.path.isfile(transcript_path):
#         transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
#         label = row['is_politics']
#         text = split_transcript_bert(transcript)
#         df_in_pol_bert.loc[index] = [label, text]
#         row['transcribed'] = 1

# for index, row in df_in_news.iterrows():
#     transcript_path = row["potentialOutPath"]
#     if os.path.isfile(transcript_path):
#         transcript = pd.read_csv(transcript_path, usecols=['start', 'end', 'content'])
#         label = row['is_news']
#         text = split_transcript_bert(transcript)
#         df_in_news_bert.loc[index] = [label, text]
#         row['transcribed'] = 1

# ds_in_pol_bert = Dataset.from_pandas(df_in_pol_bert)
# ds_in_news_bert = Dataset.from_pandas(df_in_news_bert)

### 2. Model Training! 

#### *2.1 Logistic regression*
- To optimize memory usage, I will use Keras instead of scikit-learn to build my models
- At this time, my metric of focus is accuracy

In [65]:

# Check if x_train_12_pol.shape[1] = x_train_23_pol.shape[1] = x_train_pol.shape[1]

In [68]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
log_12_pol = Sequential()
log_12_pol.add(InputLayer(input_shape=(x_12_pol.shape[1], )))
log_12_pol.add(Dense(1, activation='sigmoid'))

optimizer=tf.keras.optimizers.Adam(learning_rate=0.05)
log_12_pol.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = log_12_pol.fit(x_12_pol.toarray(), np.array(y_before_pol), epochs=50, batch_size=8, validation_split=0.3, callbacks=[callback])

#### *2.2 Bert-base*

#### *2.2.1 Tokenize*

In [87]:
ds_before_pol_bert['train'][0:2]

{'label': [1, 0],
 'text': [[['yo ancient texan sit sunday morning think first sunday may maryland rain last night yard green tree start butt']],
  [['right welcome welcome welcome talk lately may join panel panel panel']]],
 '__index_level_0__': [697, 4919]}

In [83]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_before_pol = ds_before_pol_bert.map(tokenize_function, batched=True)
tokenized_before_news = ds_before_news_bert.map(tokenize_function, batched=True)

Map:   0%|          | 0/5668 [00:00<?, ? examples/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

#### *2.2.2 Train*

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)